In [1]:
#Importing the standard libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

#Importing the solver modules
import system
from obe import obe 
from states import SigmaLevel,PiLevelParity
from obe import Excitation
import time
import warnings
warnings.filterwarnings('ignore')

In [9]:
#Create a system and generate all the states with the rotation quantum number 0 and 2, and all the -ve parity pi state with J values from 1/2 to 5/2
b=system.System([0,2],['1/2-','3/2-','5/2-'],B_field = [0.0,0.0,60.0],ignore_mF = False)


b.sigma_Hamiltonian.generate_bare()
b.sigma_Hamiltonian.Zeeman.generate_Zeeman()
b.pi_Hamiltonian.generate_bare()
b.pi_Hamiltonian.Zeeman.generate_Zeeman()

#Next diagonalize the Hamiltonian for this system
b.sigma_Hamiltonian.diagonalize()
b.pi_Hamiltonian.diagonalize()

G_global = b.sigma_Hamiltonian.diagonalized_states
GH_global = np.round(b.sigma_Hamiltonian.diagonalized_Hamiltonian,3)
E_global =b.pi_Hamiltonian.diagonalized_states
EH_global = np.round(b.pi_Hamiltonian.diagonalized_Hamiltonian,3)

print("Hamiltonian generated")

b.generate_branching_ratios(G_global,E_global)
BR_global = b.branching_ratios

Hamiltonian generated
Pi branching took : 4.006299734115601 sec
Sigma- branching took : 3.71877121925354 sec
Sigma+ branching took : 3.7109551429748535 sec


array([[0.169, 0.016, 0.054, 0.429, 0.013, 0.   , 0.018, 0.046, 0.093,
        0.021, 0.142, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.044, 0.222, 0.4  , 0.   , 0.   , 0.   , 0.053, 0.   , 0.003,
        0.104, 0.007, 0.166, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.054, 0.429, 0.169, 0.016, 0.151, 0.   , 0.093, 0.012, 0.019,
        0.042, 0.018, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.4  , 0.   , 0.044, 0.222, 0.003, 0.167, 0.003, 0.108, 0.053,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.157, 0.   , 0.376, 0.   , 0.137, 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.003, 0.028, 0.028, 0.381],
       [0.011, 0.   , 0.073, 0.068,

### BaF 138 R0 transitions

In [13]:
#Reduce the Hamiltonian to current Hilbert space
G  = G_global[0:4]
GH = GH_global[0:4,0:4]
E  = E_global[4:12]
EH = EH_global[4:12,4:12]
BR = BR_global[0:4,4:12]

In [22]:
## Drawww energy levels
plt.figure()
ground_mF = [item.states[np.argmax(np.abs(item.amplitude))].mF+np.linspace(-1/4,1/4,3) for item in G]
ground_Energy = np.array([GH[i,i] for i in range(len(G))])
ground_Energy -= np.mean(ground_Energy)
min_ground_energy = np.amin(ground_Energy)
max_ground_energy = np.amax(ground_Energy)

excited_mF = [item.states[np.argmax(np.abs(item.amplitude))].mF+np.linspace(-1/4,1/4,3) for item in E]
excited_Energy = np.array([EH[i,i] for i in range(len(E))])
excited_Energy -= np.mean(excited_Energy) - 1*np.abs(max_ground_energy-min_ground_energy)
excited_Energy += 150
          
#Plot the energy levels
for i in range(len(G)):
    plt.plot(ground_mF[i],ground_Energy[i]+np.zeros(3))
    plt.text(ground_mF[i][1],ground_Energy[i]-8,f"G[{i}]",fontsize = 8)
for j in range(len(E)):
    plt.plot(excited_mF[j],excited_Energy[j]+np.zeros(3))
    plt.text(excited_mF[j][1],excited_Energy[j]-8,f"E[{j}]",fontsize = 8)


In [30]:
## Rewrite the Hamiltonian in the current Hilbert space
H0 = np.zeros((len(G)+len(E),len(G)+len(E)),dtype=np.complex_)
H0[0:len(G),0:len(G)] = GH
H0[len(G):,len(G):] = EH
b.generate_interaction_Hamiltonian(G,E)
Hint_1=np.round(b.interaction_Hamiltonian,3)
n0 = [1/len(G)]*len(G)+[0]*len(E)


## Function to compute the integrated fluorescence
Gamma = 2*np.pi*2.7
def r22(detuning,rabi_prime): #rabi_det_params was a field earlier
    
    tsigma = 113/Gamma/4
    field = Excitation(rabi_prime,0,G[0],E[0],detuning,position=0, diameter=tsigma*4,shape = "Gaussian")

    steps=5000
    n=len(E)+len(G)
    r_init = np.zeros(np.shape(H0)).astype(np.complex128)
    for i in range(len(G)):
        r_init[i,i] = 1.0/len(G)
    
    my_obe = obe(field,[G,E],H0,Hint_1,BR)
    _,ans = my_obe.solve(steps,r_init,max_step_size = 1/Gamma,package='Python')
    
    integrated_fluorescence = 0
    for i in range(steps):
        rho_i = ans[i].reshape(n,n)
        integrated_fluorescence += np.sum(np.diag(rho_i)[len(G):n])

    print(detuning,end = ",")
    
    return np.real(integrated_fluorescence)


In [33]:
from joblib import Parallel,delayed
detuning_scan = np.arange(-300,100,2)

for rabi_prime in [2.0]:
    print(rabi_prime)
    fluorescence = Parallel(n_jobs = -1)(delayed(r22)(detuning,rabi_prime) for detuning in detuning_scan)
    array_to_write = np.array([detuning_scan,fluorescence]).T
    fname = f"BaF138_R0_rabi_{rabi_prime}.csv"
    np.savetxt(fname,array_to_write,delimiter=',')
    print("")


2.0
-294,-298,-300,-286,-290,-296,-288,-292,-282,-280,-278,-276,-272,-274,-270,-284,-268,-262,-264,-266,-260,-256,-254,-258,-252,-250,-248,-246,-242,-240,-244,-238,-236,-234,-232,-222,-226,-230,-228,-224,-220,-218,-216,-212,-206,-210,-214,-208,-204,-200,-202,-198,-194,-196,-192,-190,-188,-184,-180,-178,-176,-186,-182,-174,-172,-166,-170,-168,-162,-164,-160,-154,-158,-156,-152,-148,-150,-146,-144,-140,-142,-136,-138,-132,-134,-130,-128,-126,-124,-122,-120,-118,-114,-116,-112,-110,-108,-106,-104,-100,-98,-102,-94,-96,-92,-90,-88,-86,-84,-82,-80,-78,-74,-76,-72,-70,-66,-68,-64,-62,-60,-58,-54,-56,-52,-50,-46,-48,-42,-40,-38,-44,-36,-34,-32,-30,-18,-28,-24,-26,-22,-20,-14,-10,-8,-6,-2,-4,-12,0,-16,4,2,6,8,10,12,14,16,20,24,18,22,26,28,30,32,36,34,40,38,42,44,48,46,50,54,52,56,58,60,64,62,66,68,70,74,72,76,78,80,82,86,84,88,90,92,94,96,98,


In [35]:
plt.figure()
for rabi_prime in [2.0]:
    fname = f"BaF138_R0_rabi_{rabi_prime}.csv"
    A = np.genfromtxt(fname,delimiter = ",")
    plt.plot(A[:,0],A[:,1]/np.amax(A[:,1]))